In [3]:
import numpy as np
import pandas as pd

In [5]:
songs = pd.read_csv('data/songs.csv')
user = pd.read_csv('data/user.csv')

In [6]:
print(songs.head())
print(user.head())

               spotify_id                                               Song  \
0  7n92QzQomRCLlciO14X0kd  Nocturne en mi bÃ©mol majeur opus 9 nÂ°2: Ball...   
1  6GzMz3s0K1YKwRVI36CgRx                                       Liebestraume   
2  0P3TUNfXfhdJ4rgdZAUL6i   Ma vlast (My Fatherland): No. 2. Vltava (Moldau)   
3  3DNRdudZ2SstnDCVKFdXxG        Sonata No. 14 "Moonlight" in C-Sharp Minor"   
4  4jW2s3Spp80QQwHxdefx88  Concerto No. 1 in G Minor for Violin and Orche...   

              Artist  Rank      Genre  
0  FrÃ©dÃ©ric Chopin     1  classical  
1                NaN     2  classical  
2   BedÅ™ich Smetana     3  classical  
3                NaN     4  classical  
4                NaN     5  classical  
   ID                 Name    Genre 1    Genre 2    Genre 3  \
0   1           Sibyl Bode       rock        r&b        pop   
1   2    Eleazar Hettinger        rap  classical    hip hop   
2   3        Geovanny Lang        rap    country       soul   
3   4    Micah Satterfield 

In [ ]:
from h2ogpte import H2OGPTE

In [57]:
client = H2OGPTE(
    address='https://h2ogpte.genai.h2o.ai',
    api_key='sk-ms7SU43E34tS9UJks5RD2KM3m1JumOR2pM73Dk95VzKjM6TZ',
)

# # Chat with LLM
# chat_session_id = client.create_chat_session()
# with client.connect(chat_session_id) as session:
#     # Simple Question for Document Collection
#     answer = session.query(
#         "Can you hallucinate?",
#     ).content
#     print(answer)

# Create Collection
collection_id = client.create_collection(
    name="My first h2oGPTe collection",
    description="PDF -> text -> summary",
)

import os

# Your existing code
file_path = "/Users/blabbyduck/Desktop/Y3S2/DSA4213/LLMasters/data/user.xlsx"
with open(file_path, "rb") as f:
    user = client.upload(os.path.basename(file_path), f)

with open('/Users/blabbyduck/Desktop/Y3S2/DSA4213/LLMasters/data/listening history.xlsx', 'rb') as f:
    hist = client.upload('/Users/blabbyduck/Desktop/Y3S2/DSA4213/LLMasters/data/listening history.xlsx', f)

with open('/Users/blabbyduck/Desktop/Y3S2/DSA4213/LLMasters/data/songs_short.xlsx', 'rb') as f:
    songs = client.upload('/Users/blabbyduck/Desktop/Y3S2/DSA4213/LLMasters/data/songs_short.xlsx', f)

# # Converting the input into chunked text and embeddings...
client.ingest_uploads(collection_id, [hist,user, songs])

Job(id='d598f913-c8e7-41db-b737-e3de2ccd9b7a', name='Adding documents', passed=1.0, failed=0.0, progress=1.0, completed=True, canceled=False, date=datetime.datetime(2024, 3, 26, 16, 9, 45, tzinfo=TzInfo(UTC)), kind=<JobKind.IngestUploadsJob: 'IngestUploadsJob'>, statuses=[JobStatus(id='36fea923bf79437c86bd632a60267d92', status='Indexing done.'), JobStatus(id='f6f52470dca241bbb96b6ff5d2649e4a', status='Indexing done.')], errors=[], last_update_date=datetime.datetime(2024, 3, 26, 16, 10, 12, tzinfo=TzInfo(UTC)), duration='27s', duration_seconds=27.0)

In [59]:
chat_session_id = client.create_chat_session(collection_id)
with client.connect(chat_session_id) as session:
    # Simple Question for Document Collection
        answer = session.query(
        message="Songs that are similar to Strawberry Wine",
        rag_config={
            "rag_type": "rag",
        },
        ).content
        print(answer)


Based on the provided document, songs that are similar to "Strawberry Wine" could include:

* "Delta Dawn" by Tanya Tucker
* "Forever And Ever" by Randy Travis
* "Maybe It Was Memphis" by Pam Tillis
* "Kiss An Angel Good Mornin'" by Charley Pride
* "Bubba Shot the Jukebox" by Mark Chesnutt
* "Coal Miner's Daughter" by Loretta Lynn
* "Grandpa (Tell Me 'Bout The Good Old Days)" by The Judds
* "I Was Country When Country Wasn't Cool" by Barbara Mandrell
* "Meet Me In Montana" by Dan Seals and Marie Osmond
* "When She Cries" by The Judds
* "Pancho and Lefty" by Townes Van Zandt and Willie Nelson

These songs share similar themes of love, relationships, and nostalgia for simpler times, which are also present in "Strawberry Wine."


Prompt designs

In [ ]:
Prompt_template = """
Answer the questions based on only the following context:

{context}

---

Answer the questions based on the above context: {query}
"""
